# SimPy in <10 minutes

SimPy is a discrete-event simulation library. The behavior of active components (like vehicles, customers or messages) is modeled with **processes**. All processes live in an **environment**. They interact with the environment and with each other via **events**.

Processes are described by simple Python generators. During their lifetime, they create events and **yield** them in order to wait for them to be triggered.

When a process yields an event, the process gets suspended. SimPy resumes the process when the event occurs (we say that the event is triggered). Multiple processes can wait for the same event. SimPy resumes them in the same order in which they yielded that event.

An important event type is the `Timeout`. Events of this type are triggered after a certain amount of (simulated) time has passed. They allow a process to sleep (or hold its state) for the given time. A `Timeout` and all other events can be created by calling the appropriate method of the `Environment` that the process lives in (`Environment.timeout()` for example).

## First process

For our first process, let us consider a simple model of a supermarket (or any establishment for that matter). Customers will arrive at a supermarket and transact after some waiting. Once done, they will leave the supermarket.
This example was modified from the [Bank Renege](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html) example of SimPy

In [1]:
def customer(env):
    while True:
        print('Arrived at %d' % env.now)
        wait_duration = 15
        yield env.timeout(wait_duration)

        print('Begin transaction at %d' % env.now)
        transact_duration = 20
        yield env.timeout(transact_duration)

        print('Left at %d' % env.now)
        leave_duration = 5
        yield env.timeout(leave_duration)

Our *customer* process requires a reference to an `Environment` (env) in order to create new events. The customer’s behavior is described in an infinite loop. Remember, this function is a generator. Though it will never terminate, it will pass the control flow back to the simulation once a `yield` statement is reached. Once the yielded event is triggered (“it occurs”), the simulation will resume the function at this statement.

The *customer* repeatedly goes to and leaves the bank to perform a transaction. It announces its new state by printing a message and the current simulation time (as returned by the `Environment.now` property). It then calls the `Environment.timeout()` factory function to create a `Timeout` event. This event describes the point in time the customer arrived, left, or begins transacting. By yielding the event, it signals the simulation that it wants to wait for the event to occur.

Now that the behavior of our customer has been modeled, lets create an instance of it and see how it behaves:

In [2]:
import simpy

env = simpy.Environment()
env.process(customer(env))
env.run(60)

Arrived at 0
Begin transaction at 15
Left at 35
Arrived at 40
Begin transaction at 55


The first thing we need to do is to create an instance of `Environment`. This instance is passed into our *customer* process function. Calling it creates a process generator that needs to be started and added to the environment via `Environment.process()`.

## Resources

SimPy offers three types of resources that help you modeling problems, where multiple processes want to use a resource of limited capacity (e.g., customers at a supermarket with a limited number of checkout counters) or classical producer-consumer problems.

In this section, we’ll briefly introduce SimPy’s `Resource` class.

### Basic Resource Usage

We’ll slightly modify our customer process that we introduced in the last sections.

The customer will request to use a counter to be served. If the counter is currently in use, it waits in line until it becomes available again. Once it has paid (and is done at the counter), the customer leaves.

In [3]:
def car(env, counter, name):
    # Simulate driving to the BCS
    print(f'{name} Arrived at {env.now}')
    wait_duration = 5
    yield env.timeout(wait_duration)
    
    
    print(f'{name} Line up at {env.now}')
    with counter.request() as req:
        ## wait in line
        yield req
        
        transact_duration = 20
        yield env.timeout(transact_duration)

        print(f'{name} Left at {env.now}')
        leave_duration = 5
        yield env.timeout(leave_duration)
        

The resource’s `request()` method generates an event that lets you wait until the resource becomes available again. If you are resumed, you “own” the resource until you release it.

If you use the resource with the with statement as shown above, the resource is automatically being released. If you call `request()` without with, you are responsible to call `release()` once you are done using the resource.

When you release a resource, the next waiting process is resumed and now “owns” one of the resource’s slots. The basic Resource sorts waiting processes in a *FIFO* (first in—first out) way.

A resource needs a reference to an `Environment` and a *capacity* when it is created:

In [4]:
import simpy
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)

for i in range(4):
    env.process(car(env, counter, f'Car{i:d}'))
env.run()

Car0 Arrived at 0
Car1 Arrived at 0
Car2 Arrived at 0
Car3 Arrived at 0
Car0 Line up at 5
Car1 Line up at 5
Car2 Line up at 5
Car3 Line up at 5
Car0 Left at 25
Car1 Left at 50
Car2 Left at 75
Car3 Left at 100
